# `Scraper` de `Datos Editorial UPTC`

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import re

## `Scraper` de `URLs` 

In [ ]:
#url de libros Uptc
url = 'https://editorial.uptc.edu.co/catalogo.html?product_list_limit=60'
bucle = 'https://editorial.uptc.edu.co/catalogo.html?p=2&product_list_limit=60'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html')


scrap = soup.find('div',class_="sa-product-container-v3 grid")
productos_url = scrap.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")
url_libros = [url.find('a')["href"] for url in productos_url]

urls = pd.DataFrame(url_libros,columns=['url'])

for i in range(2,6):
    url = f'https://editorial.uptc.edu.co/catalogo.html?p={i}&product_list_limit=60'
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html')
    productos_url = soup.find('div',class_="sa-product-container-v3 grid").find_all('li',class_="sa-product-item-v3 sa-product_grid_size")
    url_libros = [url.find('a')["href"] for url in productos_url]
    urls = pd.concat([urls,pd.DataFrame(url_libros,columns=['url'])])


In [ ]:
#urls.to_json('urls.json',index=False)
urls.shape

In [2]:
url_libros = pd.read_csv('urls.csv')

In [64]:
def extraer_palabras_clave(texto):
    # Patrón para encontrar palabras clave y sus valores
    patron = r'([^:]+):\s*([^\n]+)'
    # Encontrar todas las coincidencias
    coincidencias = re.findall(patron, texto)
    
    # Crear diccionario con las palabras clave
    palabras_clave = {}
    for clave, valor in coincidencias:
        # Limpiar espacios en blanco
        clave = clave.strip()
        valor = valor.strip()
        palabras_clave[clave] = valor
    
    return palabras_clave

def extraer_info_colaborador(bloque):
    # Nombre invertido
    nombre_invertido = bloque.split('\n')[0].strip()
    
    # ORCID
    orcid_match = re.search(r'(https://orcid\.org/[^\s]+)', bloque)
    orcid = orcid_match.group(1) if orcid_match else None
    
    # Afiliación
    afiliacion_match = re.search(r'Afiliación\s*\n([^\n]+)', bloque)
    afiliacion = afiliacion_match.group(1).strip() if afiliacion_match else None
    
    # Biografía
    bio_match = re.search(r'Biografía:\s*([\s\S]+?)(?:\n\S|$)', bloque)
    if bio_match:
        biografia = bio_match.group(1).strip()
    else:
        # Si no encuentra el patrón, toma todo después de "Biografía:"
        bio_idx = bloque.find("Biografía:")
        biografia = bloque[bio_idx+10:].strip() if bio_idx != -1 else None
    
    # Email (opcional)
    email_match = re.search(r'[\w\.-]+@[\w\.-]+', bloque)
    email = email_match.group(0) if email_match else None

    return {
        "nombre_invertido": nombre_invertido,
        "orcid": orcid,
        "afiliacion": afiliacion,
        "biografia": biografia,
        "email": email
    }

 

In [68]:
temp_datos = []
for i in range(1):
    titulos = []
    autores = []
    palabras_clave = []
    precios = []
    descripcion = []


    res = requests.get(url_libros['url'][i])
    soup = BeautifulSoup(res.text,'html')
    time.sleep(5)
    scrap=soup.find('div',class_="product-info-main")
    titulos.append(scrap.find('div',class_="col-xs-12").find('h2').text)
    ## busquemos lo autores
    temp = [i for i in scrap.find_all('div',class_="col-xs-12")]
    temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_author' in a['href'] ]
    autores.append(temp1)
    # busca las palabras clave
    temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' in a['href'] ]
    palabras_clave.append(temp1)
    # DOi
    try:
        temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' not in a['href'] and  'products_author' not in a['href']]
        if temp1 != []:
            doi = temp1
        else :
            doi = None 
    except Exception as e:
        print(f'No se encontro doi: {e}')
        doi = None

    # precios
    try:
        temp = [a for a in scrap.find_all('div',class_='price-box price-final_price')]
        if temp:  # Si hay elementos de precio
            temp1 = [a.text.strip() for a in temp]
            precios.append(temp1)
        else:  # Si no hay elementos de precio
            precios.append(['Descargable gratis'])
    except Exception as e:
        print(f'No se encontro precio: {e}')
        precios.append(['Descargable gratis'])
    # descripcion
    descripcion.append(scrap.find('div',class_='description-wrapper').text)
    # eBook
    try:
        scrap = soup.find('div',class_="product-info-metadata")
        metadata_ebook = extraer_palabras_clave(scrap.text)
    except Exception as e:
        print(f'No se encontro metadata_ebook: {e}')
        metadata_ebook = None
    #impreso
    try:
        scrap = soup.find_all('div',class_="product-info-metadata")[1]
        metadata_imp = extraer_palabras_clave(scrap.text)
    except Exception as e:
        print(f'No se encontro metadata_imp: {e}')
        metadata_imp = None
    #clasificacion de tematica
    dato = soup.find('div','item content').text.replace('\n','')
    clasificacion_tematica = dato
    #tabla de contenido
    a=soup.find('div',class_='product data items').text.find('Tabla de contenido')
    b=soup.find('div',class_='product data items').text.find('Colaboradores')
    dato = soup.find('div',class_='product data items').text
    if a != -1:
        tabla_contenido=dato[a:b]
    else:
        tabla_contenido= None
    ## informacion de autores
    a=soup.find('div',class_='product data items').text.find('Colaboradores')
    dato = soup.find('div',class_='product data items').text
    colaboradores_raw = dato[a:].split('Nombre invertido')[1:]
    colaboradores = []
    for bloque in colaboradores_raw:
        info = extraer_info_colaborador(bloque)
        colaboradores.append(info)

    datos = {
            "title": titulos,
            "colaboradores":autores,
            "precio":precios,
            "palabras clave":palabras_clave,
            "doi":doi,
            "descripcion":descripcion,
            "metadata_ebook":metadata_ebook,
            "metadata_imp":metadata_imp,
            "clasificación_temática":clasificacion_tematica,
            'tabla_contenido':tabla_contenido,  
            'inf_autores' :colaboradores 
            }
    temp_datos.append(datos)
    

In [70]:
with open('datos.json', mode='w', encoding='utf-8') as f:
   json.dump(temp_datos, f, ensure_ascii=False, indent=4)


In [65]:
# Inspeccionar la estructura de los datos

a=soup.find('div',class_='product data items').text.find('Colaboradores')
dato = soup.find('div',class_='product data items').text
colaboradores_raw = dato[a:].split('Nombre invertido')[1:]
colaboradores = []
for bloque in colaboradores_raw:
    info = extraer_info_colaborador(bloque)
    colaboradores.append(info)

In [69]:
temp_datos

[{'title': ['Transporte de fluidos bombas centrífugas'],
  'colaboradores': [['Otto Caro Niño', 'Orlando Díaz Parra']],
  'precio': [['', 'COP $30.000', 'COP $40.000']],
  'palabras clave': [['Fluido',
    'Sistema de bombeo',
    'Bomba centrífuga',
    'Mecánica de fluidos',
    'Transporte de fluidos',
    'Operación y Mantenimiento']],
  'doi': ['10.19053/uptc.9789586609265'],
  'descripcion': ['\nUn sistema compuesto por una conducción y sus accesorios, por el que se transporta un fluido presurizado y en el que un dispositivo denominado “bomba hidráulica” interviene para hacer transitar el fluido, se denomina sistema de bombeo. Por sistemas de bombeo se transporta, entre otros, agua para riego, en plantas de potabilización y tratamiento, en redes de distribución de agua potable e industrial, así como también en sistemas de enfriamiento y calentamiento. Industrialmente se transporta, además de agua, leche y sus derivados, jugos y alimentos, muy diversos tipos de aceites y combustib

In [26]:
scrap = soup.find_all('div',class_="product data items")

In [30]:
scrap.find('div',{'rolo':'tablist'}) 
# consejo de facultada

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [45]:
item title active

SyntaxError: invalid syntax (2855226362.py, line 1)

In [22]:
scrap = soup.find('div',class_="product data items")
temp1 = [a.text for a in scrap.find_all('span')]


In [47]:
[]==[]

True

In [52]:
print(soup.find('div',class_='product data items').find('div',{'class':'item content'}).text.replace('\n',''))

BISACPHI000000 FILOSOFÍA > GeneralTHEMAQDT Temas de la filosofíaDEWEY107 Filosofía y psicología > Generalidades > Educación, investigación en filosofía


In [58]:
temp_datos[-1]

{'title': ['Gestión Educativa, Perspectivas, Actores y Contextos'],
 'colaboradores': [['Liliana Inés Ávila Garzón',
   'Rafael Enrique Buitrago Bonilla',
   'María Rosa Estupiñán Aponte',
   'Adriana María Cárdenas Casas',
   'Alex Oswaldo Sánchez Huarcaya',
   'Alexandra Inés Puentes Suárez',
   'Angie Lorena Ariza Velasco',
   'Diana Margot Guerrero Galarza',
   'Esmeralda Díaz López',
   'Kassandra Verastegui Vives',
   'Nidia Marcela García Gómez',
   'Ruby Paola Villavicencio Gastelu',
   'Ruth Nayibe Cárdenas Soler',
   'Sandra Milena Pinzón Jaime']],
 'precio': [['', 'COP $70.000']],
 'palabras clave': [['Gestión Educativa',
   'Emociones',
   'Maestros',
   'Memorias de Gestores',
   'Cultura Escolar',
   'Contextos']],
 'doi': '10.19053/uptc.9789586608350',
 'descripcion': ['\nGestión Educativa. Perspectivas, Actores y Contextos es un libro imprescindible para aquellos interesados en ahondar en los procesos de gestión en el ámbito educativo. A través de los distintos capítulo

In [53]:
temp1 = [a.text for a in scrap.find_all('strong')]
temp2 = [a.text for a in scrap.find_all('span')]
metadata_imp ={c:v for c,v in zip(temp1,temp2)}

In [54]:
metadata_imp

{'DOI:': 'Tapa blanda o Bolsillo',
 'Detalle de formato:': 'Activo',
 'Estado:': '9789586609265',
 'ISBN-13:': 'Texto (legible a simple vista)',
 'Tipo de contenido principal:': 'Español',
 'Idioma del texto:': '14 x 21  x 0.6',
 'Tamaño (cm):': '106 Páginas',
 'Número de páginas del contenido principal:': 'universidad Pedagógica y Tecnológica de Colombia',
 'Sello editorial: ': 'Nueva edición',
 'Tipo de edición: ': '1',
 'Número de edición: ': 'Tunja',
 'Ciudad de publicación:': 'Colombia',
 'País de publicación:': '2025',
 'Fecha de publicación:': 'Exclusivo para un punto o canal de venta',
 'Tipo de restricción de venta:': '419307',
 'SIIEL SKU:': 'COP $40.000'}

In [44]:
metadata_ebook

{'Detalle de formato:': 'Digital (suministrado electrónicamente)',
 'Formato:': 'EPUB',
 'Estado:': 'Activo',
 'ISBN-13:': '9789586609272',
 'Tipo de contenido del producto:': 'Texto (legible a simple vista)',
 'Protección técnica de publicación digital:': 'DRM',
 'Tipo de restricción de uso en publicaciones digitales:': 'Share',
 'Permiso de uso en publicaciones digitales:': 'Permitido, con limitaciones',
 'Idioma del texto:': 'Español',
 'Tamaño del archivo:': '1,593 Kbytes',
 'Sello editorial: ': 'Universidad Pedagógica y Tecnológica de Colombia',
 'Tipo de edición: ': 'Nueva edición',
 'Número de edición: ': '1',
 'Ciudad de publicación:': 'Tunja',
 'País de publicación:': 'Colombia',
 'Fecha de publicación:': '2025',
 'Tipo de restricción de venta:': 'Exclusivo para un punto o canal de venta',
 'Precio: ': 'COP $30.000'}

In [41]:
len(temp1) ,len(temp2)

(18, 18)

In [39]:
for a in scrap.find_all('span'):
    print(a.text)

Digital (suministrado electrónicamente)
EPUB
Activo
9789586609272
Texto (legible a simple vista)
DRM
Share
Permitido, con limitaciones
Español
1,593 Kbytes
Universidad Pedagógica y Tecnológica de Colombia
Nueva edición
1
Tunja
Colombia
2025
Exclusivo para un punto o canal de venta
COP $30.000


In [38]:
temp

['Detalle de formato:',
 'Formato:',
 'Estado:',
 'ISBN-13:',
 'Tipo de contenido del producto:',
 'Protección técnica de publicación digital:',
 'Tipo de restricción de uso en publicaciones digitales:',
 'Permiso de uso en publicaciones digitales:',
 'Idioma del texto:',
 'Tamaño del archivo:',
 'Sello editorial: ',
 'Tipo de edición: ',
 'Número de edición: ',
 'Ciudad de publicación:',
 'País de publicación:',
 'Fecha de publicación:',
 'Tipo de restricción de venta:',
 'Precio: ']

In [115]:
temp1

['Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento']

In [73]:
temp1

['Otto Caro Niño',
 'Orlando Díaz Parra',
 'Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento',
 '10.19053/uptc.9789586609265']

In [9]:
res.status_code

200

 tenemos un estatus 200 lo cual es correcto la solicitud
 

In [4]:
soup = BeautifulSoup(res.text,'html')

In [19]:
soup.find('title').get_text()

'Catálogo - UPTC'

In [22]:
datos=soup.find('div',class_="sa-product-container-v3 grid")


In [26]:
productos = datos.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

In [ ]:
a = {
    titulo: 
    colaboradores:
    ano:
    precio:
}

In [48]:
title =[i.find('span').get_text(strip=True) for i in productos]

In [62]:
ano = [i.find('div',class_='product-list-edition').get_text(strip=True) for i in productos]

In [76]:
productos[].find('span',class_='minimal-price').get_text()

'\nImpreso\n\n\nCOP $40.000\n\n\n'

In [101]:
datos = []
for i in productos:
    a = {
        "title":i.find('span').get_text(strip=True),
        "colaboradores":list(i.find('div',class_='sa-product-author-v3').stripped_strings)[1],
        "precio": i.find('span',class_='minimal-price').get_text().replace("\n",""),
        "ano": i.find('div',class_='product-list-edition').get_text(strip=True)
    }
    datos.append(a)


In [102]:
datos

[{'title': 'Transporte de fluidos bombas centrífugas',
  'colaboradores': 'Otto Caro Niño                                                                                     y otros',
  'precio': 'eBookCOP $30.000',
  'ano': '2025'},
 {'title': 'Introducción al cálculo de probabilidades para estudiantes de ingeniería',
  'colaboradores': 'Alex L. Rojas',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Derechos colectivos en Colombia:',
  'colaboradores': 'Luis Bernardo Díaz Gamboa                                                                                     y otros',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Resistencias y voces de las mujeres:',
  'colaboradores': 'Celina de Jesús Trimiño Velásquez                                                                                     y otros',
  'precio': 'ImpresoCOP $40.000',
  'ano': '2025'},
 {'title': 'Manual de prácticas pedagógicas para la formación de maestros.',
  'colaboradores': 'Myriam Cecilia